In [2]:
import fiddler as fdl

# Initialize Fiddler client
fdl.init(url="https://fiddler.servicenow.com", token="swP24N9AZdX5oCMzVbWXOlWOgdrLbk4q4GogO7IZWoc")

# List of all percentage-based data integrity metric IDs
PERCENTAGE_METRIC_IDS = [
    'null_violation_percentage',
    'type_violation_percentage',
    'range_violation_percentage',
    'any_violation_percentage'
]

# Get all projects
projects = list(fdl.Project.list())
print(f"Found {len(projects)} projects")

# Initialize containers for all alerts
all_alerts = []
integrity_alerts = []

# Loop through each project to get models
for project in projects:
    project_name = project.name
    print(f"Processing project: {project_name}")
    
    # Get all models in this project
    models = list(fdl.Model.list(project_id=project.id))
    print(f"  Found {len(models)} models in project {project_name}")
    
    # Loop through each model to get alerts
    for model in models:
        model_name = model.name
        print(f"    Processing model: {model_name}")
        
        # Get all alerts for this model
        try:
            model_alerts = list(fdl.AlertRule.list(model_id=model.id))
            all_alerts.extend(model_alerts)
            print(f"      Found {len(model_alerts)} alerts for model {model_name}")
        except Exception as e:
            print(f"      Error retrieving alerts for model {model_name}: {str(e)}")
            continue

print(f"Found {len(all_alerts)} total alerts across all models")

# Filter to data integrity alerts with percentage metrics
integrity_alerts = [alert for alert in all_alerts 
                   if alert.metric_id in PERCENTAGE_METRIC_IDS]
print(f"Found {len(integrity_alerts)} data integrity percentage alerts to modify")


Found 80 projects
Processing project: ml_agg_prd
  Found 1 models in project ml_agg_prd
    Processing model: ml_agg_prd_model_1
      Found 91 alerts for model ml_agg_prd_model_1
Processing project: ml_agg_prd_v2
  Found 2 models in project ml_agg_prd_v2
    Processing model: ml_agg_prd_v2_model_1
      Found 92 alerts for model ml_agg_prd_v2_model_1
    Processing model: ml_agg_prd_v2_model_2
      Found 96 alerts for model ml_agg_prd_v2_model_2
Processing project: gam_prd
  Found 1 models in project gam_prd
    Processing model: gam_prd_model_1
      Found 250 alerts for model gam_prd_model_1
Processing project: itsm_risk
  Found 1 models in project itsm_risk
    Processing model: itsm_risk_model_10
      Found 32 alerts for model itsm_risk_model_10
Processing project: ml_agg_prd_v4
  Found 0 models in project ml_agg_prd_v4
Processing project: lead_scoring
  Found 1 models in project lead_scoring
    Processing model: lead_scoring_model_1
      Found 92 alerts for model lead_scoring

In [ ]:

# Counters
raw_value_count = 0
time_period_count = 0
modified_count = 0

# Process each alert
for alert in integrity_alerts:
    print(f"Processing alert: {alert.name} (ID: {alert.id})")
    
    # Check the comparison type
    if alert.compare_to == fdl.CompareTo.TIME_PERIOD:
        time_period_count += 1
        print(f"  Skipping time period comparison alert")
        continue
    
    if alert.compare_to == fdl.CompareTo.RAW_VALUE:
        raw_value_count += 1
        modified = False
        
        # Check and update critical_threshold
        if alert.critical_threshold is not None and 0 < alert.critical_threshold < 1:
            old_threshold = alert.critical_threshold
            alert.critical_threshold = old_threshold * 100
            print(f"  Updating critical threshold from {old_threshold} to {alert.critical_threshold}")
            modified = True
        
        # Check and update warning_threshold
        if alert.warning_threshold is not None and 0 < alert.warning_threshold < 1:
            old_threshold = alert.warning_threshold
            alert.warning_threshold = old_threshold * 100
            print(f"  Updating warning threshold from {old_threshold} to {alert.warning_threshold}")
            modified = True
        
        # Update the alert if changes were made
        if modified:
            alert.update()
            modified_count += 1
            print(f"  Alert updated successfully")
        else:
            print(f"  No changes needed for this alert (thresholds already in correct format)")
    else:
        print(f"  WEIRD ALERT COMPARE TYPE: {alert.compare_to}")

print(f"Summary:")
print(f"- Total integrity percentage alerts: {len(integrity_alerts)}")
print(f"- Raw value comparison alerts: {raw_value_count}")
print(f"- Time period comparison alerts: {time_period_count}")
print(f"- Alerts modified: {modified_count}")